In [2]:
import pandas as pd
import numpy as np

Счачиваем данные

In [87]:
df = pd.read_csv('https://raw.githubusercontent.com/Steam-Better-RecSys/Steam-Better-RecSys-ML/main/Data/steam_wizzard_data.csv', index_col=0)

df.columns

Index(['App ID', 'Title', 'Unfiltered Reviews Total', 'Release Date', 'Tags',
       'name_slug', 'Description'],
      dtype='object')

Оставляем только нужные нам данные

In [88]:
df = df.loc[:, ['App ID', 'Tags']]

Взрывваем тэги, чтобы с ними было удобнее работать

In [89]:
df['Tags'] = df['Tags'].str.split(',')
df = df.explode('Tags')
df['Tags'] = df['Tags'].apply(lambda x: x.lstrip().rstrip())

Стоит обратить внимание, что в тэгах есть `Football (Soccer)` и `Soccer` - их стоит объединить просто в `Soccer`.

In [90]:
df.loc[df['Tags'] == 'Football (Soccer)', 'Tags'] = 'Soccer'

Получаем уникальные тэги

In [91]:
tags = df['Tags'].unique()

Обратный взрыв

In [94]:
df = df.groupby(['App ID'])['Tags'].apply(lambda x: (','.join(x)).split(',')).reset_index()

Добавляем в df свой столбец на каждый из тэгов

In [ ]:
for tag in tags:
  df[tag] = 0

Рассчитываем вес тэга, есть 2 гипотезы:
1. чем ближе тэг к начале списка, тем он важнее;
2. чем ближе тэг к концу списка, тем он важнее.
получается параболическая зависимость веса тэга от его номера (индекса) в списке тэгов

In [97]:
def tag_weight(tags, tag):
  if tag in tags:
    i = tags.index(tag)
    l = len(tags)
    if l - i < i:
      i = l - i
    return max(1 - 0.1 * i, 0.1)
  return 0

In [ ]:
for tag in tags:
  df[tag] = df['Tags'].apply(lambda tags: tag_weight(tags, tag))

Проверяем

In [ ]:
df.loc[df['Moddable'] == 1]

In [ ]:
df.loc[df['Moddable'] == 0.9]

Ставим нужный `index`

In [103]:
df = df.set_index('App ID')

Удаляем колонку `Tags`, так как она не будет использоваться в проде

In [105]:
df = df.drop(columns=['Tags'])

Сохраняем данные

In [107]:
df.to_csv('steam_wizzard_tags_data.csv')